In [1]:
import sys, os
sys.path.append(os.path.expanduser('~/Project/source'))
#
import numpy as np
import importlib, os, hashlib, json, copy, time
from quick_sim import setup_sim
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_pot, coupled_flux_qubit_force, coupled_fq_pot
import edward_tools.cfq_batch_sweep as cfq_batch_sweep
from edward_tools.visualization import animate_sim_flux_qubit
import kyle_tools as kt
import matplotlib.pyplot as plt
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
import edward_tools.visualization as visualization
import matplotlib.animation as animation

class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
create_system_from_storage_and_computation_protocol = coupled_fq_protocol_library.create_system_from_storage_and_computation_protocol
create_system = coupled_fq_protocol_library.create_system

# print(PHI_0, _lambda, _theta, _eta,)
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c0]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

coupled_fq_pot = Potential(coupled_flux_qubit_pot, coupled_flux_qubit_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

# gamma, beta_1, beta_2 = 0, 0, 0
gamma = 20
# d_beta_1, d_beta_2 = 0.6, 0.6
d_beta_1, d_beta_2 = 0, 0
params['sim_params'] = [_lambda, _theta, _eta]

N =  params['N'] = 5_00
params['dt'] = 1/1_00
params["comment"] = "test_subdivide_N"

max_n_per_batch = 1_00
batch_list = []

while N > max_n_per_batch:
    batch_list.append(max_n_per_batch)
    N -= max_n_per_batch
batch_list.append(N)

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx,  "phi_2_dcx": phi_2_dcx,
        "M_12": 0, 'x_c': x_c
}

protocol_list = [
    {
        "duration": 10/time_scale_factor , "phi_2_dcx": 3/time_scale_factor, "name": "mix in y direction"
    },

    {"duration": 18/time_scale_factor, "M_12": -0.9, "name": "conditional tilt"}, # conditional tilt

    {"duration": 16/time_scale_factor, "phi_2_dcx": 0, "name": "raise the barrier"}, # raise the barrier

    {"duration": 12/time_scale_factor, "phi_1_dcx": 3/time_scale_factor, "name": "mix in x direction"}, # mix in x direction

    {"duration": 50/time_scale_factor, "phi_1_dcx": 0, "M_12": 0, "name": "4 well potential "}, # 4 well potential
]
"""
# step 3: create the relevant storage protocol and computation protocol
"""

# check if the simulation is a sweep or not
def multiprocessing_worker_function(_N):
    # print(protocol_list)
    _params = copy.deepcopy(params)
    _params["N"] = _N
    computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, protocol_list)
    storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

    """
    # step 4: create the coupled_fq_runner
    """
    cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = _params, storage_protocol= storage_protocol, computation_protocol= comp_protocol)
    cfqr.initialize_sim()
    cfqr.set_sim_attributes()
    init_state_saved = cfqr.init_state

    """
    step 5: Run sweep simulation in parallel
    """
    manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]
    params['sim_params'] = [_lambda, _theta, _eta]

    """
    step 5a: single simulation
    """
    simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(_params, initial_parameter_dict, protocol_list, initial_state = init_state_saved, manual_domain = manual_domain, phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, frameRate = -1)
    #
    # cfqr = simResult["cfqr"]
    # cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True)
    # simResult["cfqr"] = 0
    simResult["ani"] = 0



    return simResult

from multiprocessing import Process, Pool
if __name__ == "__main__":  # confirms that the code is under main function
    p = Pool(processes=len(batch_list))
    dataArray = p.map(multiprocessing_worker_function, batch_list)
    p.close()
    print("finish all")
    print(dataArray)

    parameterArray = np.array([item["params"] for item in dataArray])
    np.save("parameterArray.npy", parameterArray)

    # new simualtion data
    new_simulation_id = hashlib.sha256(bytes(str(time.time()), encoding='utf8')).hexdigest()
    simulation_data = dataArray[0]["simulation_data"]
    simulation_data["simulation_id"] = new_simulation_id
    simulation_data["simulation_date"] = str(simulation_data["simulation_date"])

    n_array = np.array([len(item["work_distribution"]) for item in dataArray])
    total_N = np.sum(n_array)



    # combine animation
    ani_max_N = 10000
    if total_N < ani_max_N:
        sampleNumber = n_array
    else:
        sampleNumber = n_array / total_N * ani_max_N

    sampleRatio = np.repeat(1, len(n_array))
    sampleTrajectoryArray = [data["cfqr"].sim.output.all_state["states"] for data in dataArray]
    trajectoryNumber = np.array([len(_t) for _t in sampleTrajectoryArray])
    sampleNumber = trajectoryNumber * sampleRatio
    sampleIndexArray = [np.random.randint(int(tn), size=int(sn)) for tn, sn in zip(trajectoryNumber, sampleNumber)]

    allSampleTrajectory = np.concatenate(sampleTrajectoryArray)

    # animations
    vmin, vmax = 0, 100
    frameRate = 10

    manual_domain=[[-5, -5], [5, 5]]
    time_range = [0, np.sum([item["duration"] for item in dataArray[0]["protocol_list_item"]])]
    cfqr = dataArray[0]["cfqr"]

    ani,_,_ = visualization.animate_sim_flux_qubit(allSampleTrajectory, system = cfqr.system ,
                        times = time_range, frame_skip=frameRate, color_by_state=True,
                                                       vmin = vmin, vmax = vmax,
                                                       manual_domain = manual_domain)

    FFwriter = animation.FFMpegWriter(fps=10)

    # video = ani.to_html5_video()
    # html = display.HTML(video)
    # display.display(html)

    ani.save(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{new_simulation_id}_szilard_engine.mp4', writer = FFwriter)
    # simResult["ani"].save(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{simResult["simulation_data"]["simulation_id"]}_szilard_engine.mp4', writer = FFwriter)

    work_distribution_array =  [item["work_distribution"] for item in dataArray]
    [np.save(f"work_distribution_{i}.npy", wd) for i, wd in enumerate(work_distribution_array)]

    combinedWorkDistributionArray = np.concatenate(work_distribution_array)
    np.save("combinedWorkDistributionArray.npy", combinedWorkDistributionArray)

    # *** jarzynski_term ***
    new_jarzynski_term = np.mean(np.exp(-combinedWorkDistributionArray))

    # save figure
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.hist(combinedWorkDistributionArray, bins = 30)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, f"Jarzyn: {new_jarzynski_term: .3g}", transform=ax.transAxes, fontsize=14, verticalalignment='top',bbox=props)
    # plt.hist(work_distribution)
    # if save:
        # np.save(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{new_simulation_id}_work_distribution.npy', work_distribution)
    plt.savefig(f'coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{new_simulation_id}_work_distribution.png')

    # *** work statistics ***
    work_statistic_array =  np.array([item["work_statistic"]  for item in dataArray])
    [np.save(f"work_statistic_{i}.npy", ws) for i, ws in enumerate(work_statistic_array)]

KeyboardInterrupt: 

In [60]:
import numpy as np
n_array = np.array([100, 100, 100, 100, 100] )
total_N = np.sum(n_array)

In [61]:
ratio_array = n_array / total_N
# ratio_array_n_by_2 = np.concatenate([ratio_array, ratio_array], axis = 1)

In [65]:
randomArray = np.random.rand(5, 10, 2)

In [67]:
for r in zip(ratio_array, randomArray):
    
    print(r)

(0, (0.2, array([[0.82302088, 0.37792811],
       [0.74323094, 0.67941961],
       [0.65243223, 0.62895429],
       [0.87862748, 0.5830489 ],
       [0.26722581, 0.02221184],
       [0.15070991, 0.39683946],
       [0.37307021, 0.88750885],
       [0.99548891, 0.84159908],
       [0.50615261, 0.79539292],
       [0.40791089, 0.26690216]])))
(1, (0.2, array([[0.89063073, 0.40475457],
       [0.66734177, 0.42056628],
       [0.48615172, 0.26473204],
       [0.66329613, 0.2524    ],
       [0.04438459, 0.54642439],
       [0.20692022, 0.30226253],
       [0.67789106, 0.46103   ],
       [0.58911298, 0.87573684],
       [0.87045144, 0.48951049],
       [0.04214462, 0.65507193]])))
(2, (0.2, array([[0.29345633, 0.2983669 ],
       [0.10941158, 0.25864589],
       [0.11627458, 0.86697051],
       [0.06004838, 0.89951133],
       [0.60900275, 0.29980687],
       [0.35945286, 0.78471413],
       [0.93041229, 0.57578095],
       [0.01632424, 0.4869603 ],
       [0.35312515, 0.36824641],
       

In [58]:
dataArray.shape[0]

5

In [69]:
ratio_array_n_by_2

array([[0.2, 0.2],
       [0.2, 0.2],
       [0.2, 0.2],
       [0.2, 0.2],
       [0.2, 0.2]])

In [72]:
randomArray = np.random.rand(5, 10, 2)

In [76]:
weightedArray = [np.multiply(item,  r)  for r, item in zip(ratio_array, randomArray)]

In [81]:
weightedArray

[array([[0.07366666, 0.09857044],
        [0.06948778, 0.1969857 ],
        [0.16301708, 0.07239084],
        [0.18795541, 0.12471493],
        [0.10681681, 0.18938969],
        [0.08332576, 0.18440712],
        [0.1651833 , 0.14073937],
        [0.08865421, 0.0611399 ],
        [0.14838349, 0.1089557 ],
        [0.07523064, 0.15465027]]),
 array([[0.03395744, 0.19815035],
        [0.17919476, 0.08097432],
        [0.03148991, 0.1368209 ],
        [0.01750354, 0.01709535],
        [0.01626561, 0.05911781],
        [0.16123882, 0.04369223],
        [0.08557889, 0.07902533],
        [0.02480763, 0.08064676],
        [0.11390643, 0.08049003],
        [0.12375822, 0.18368592]]),
 array([[0.14658694, 0.03574438],
        [0.1937319 , 0.10601792],
        [0.01943927, 0.00481311],
        [0.09637938, 0.00039337],
        [0.00129655, 0.05130162],
        [0.00620846, 0.17746323],
        [0.03869769, 0.09279964],
        [0.10827713, 0.06017768],
        [0.19426706, 0.13737161],
        [0

In [79]:
np.sum(weightedArray, axis = 0)

array([[0.4430101 , 0.51611561],
       [0.57342347, 0.52261103],
       [0.34527148, 0.38190263],
       [0.50097217, 0.31939006],
       [0.25787602, 0.52638837],
       [0.50866123, 0.61048973],
       [0.41640465, 0.61560012],
       [0.43356296, 0.28982692],
       [0.5400866 , 0.45055654],
       [0.34260379, 0.61909474]])

In [75]:

np.sum([np.multiply(item,  r)  for r, item in zip(ratio_array, randomArray)], axis = 1)

array([[1.16172115, 1.33194396],
       [0.78770125, 0.959699  ],
       [0.80786435, 0.82048152],
       [0.80989839, 0.83807848],
       [0.79468734, 0.90177276]])

In [24]:
ratio

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [28]:
ratio = n_array / total_N
[np.multiply(item,  r)  for r, item in enumerate(zip(ratio, np.random.rand(5, 10, 2)))]

<ipython-input-28-e1f79c12c1fb>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  [np.multiply(item,  r)  for r, item in enumerate(zip(ratio, np.random.rand(5, 10, 2)))]


[array([0.0, array([[0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.],
                    [0., 0.]])], dtype=object),
 array([0.2, array([[0.37171702, 0.6134067 ],
                    [0.68737951, 0.96174691],
                    [0.33428527, 0.72817671],
                    [0.16016573, 0.67197895],
                    [0.87761866, 0.123189  ],
                    [0.37640925, 0.98097873],
                    [0.75657258, 0.25864431],
                    [0.13072283, 0.25523755],
                    [0.0250623 , 0.33669778],
                    [0.37221785, 0.18202753]])], dtype=object),
 array([0.4, array([[0.39831568, 0.22338926],
                    [0.1229712 , 1.95080785],
                    [1.38680192, 1.55109812],
                    [1.45513283, 1.59065577],
                    

In [14]:
np.random.rand(5, 10, 2)

array([[[0.85400075, 0.78634171],
        [0.45509725, 0.63652105],
        [0.8913475 , 0.11527186],
        [0.58285505, 0.64004218],
        [0.89867315, 0.3880855 ],
        [0.61098873, 0.98322279],
        [0.52289958, 0.01419751],
        [0.12435054, 0.56492711],
        [0.95748913, 0.30056516],
        [0.14160116, 0.5954643 ]],

       [[0.9886705 , 0.27406434],
        [0.26690382, 0.24357805],
        [0.12867269, 0.5128755 ],
        [0.9763045 , 0.98164568],
        [0.69859437, 0.69141897],
        [0.18397591, 0.13360821],
        [0.80975274, 0.12256251],
        [0.87565934, 0.62978503],
        [0.54791902, 0.78630903],
        [0.15855284, 0.28962583]],

       [[0.87500189, 0.09042574],
        [0.18498994, 0.10896876],
        [0.68003709, 0.07863994],
        [0.19250184, 0.27676071],
        [0.55200821, 0.67178085],
        [0.60017292, 0.2017931 ],
        [0.86787603, 0.67617656],
        [0.60895787, 0.0779123 ],
        [0.98742463, 0.69033407],
        [0

In [2]:
import os

# upload file to server through ssh

In [114]:
fileNameArray = [
#                  "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/6ba93082ae3f794e8e44ec11a9d40878e864e525dd9732ac9fda2337f3c7f8c1_final_state.npy",
#                 "parallel_cfqr_parameter_sweep.py",
                "parallel_cfqr_N_subdivide.py",
                "PARAMETER_INPUT.py",
#                 "edward_tools/cfq_batch_sweep.py"
]

for fileName in fileNameArray:
    command = f"scp /Users/edwardtang/Project/source/{fileName} tkwtang@mothra.csc.ucdavis.edu:source/{fileName}"
    os.system(command)

## zip data of the gallery

In [3]:
command = f"ssh -t Mothra 'cd source; bash -s zip_data.sh'"
os.system(command)

65280

## update the local machine gallery

In [5]:
command = f"python update_gallery.py"
os.system(command)

0